<a href="https://colab.research.google.com/github/LeidyDahian/kaggle/blob/main/03%20-%20modelo%20con%20preprocesado%20por%20moda%20y%20red%20neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
# Carga el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Saving kaggle.json to kaggle.json
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.12GB/s]
Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout


In [ ]:
test_df = pd.read_csv('/content/test.csv', encoding= 'utf-8')
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')

# Preprocesado
El preprocesamiento implementado resultó ser muy efectivo, manteniendo la limpieza de la columna PROGRAMA como en versiones anteriores. En este caso, las columnas categóricas y numéricas con valores faltantes fueron completadas utilizando la moda de cada una. Además, se eliminaron columnas irrelevantes y duplicadas como ESTU_PRIVADO_LIBERTAD y la repetida FAMI_TIENEINTERNET.1. Para facilitar su manipulación, se renombraron las columnas, se convirtieron los valores booleanos a binarios, y se aplicó codificación One-Hot a todas las variables categóricas seleccionadas. Finalmente, la variable RENDIMIENTO_GLOBAL fue transformada a un valor ordinal, permitiendo su uso en modelos supervisados que requieren variables numéricas.

In [ ]:
category_cols = [i for i in df.columns if not i in df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))


def Preprocesar(df):
  def obtener_primera_frase(cadena):
    return cadena.split(' ', 1)[0]

  def quitar_tildes(palabra):
    return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(obtener_primera_frase)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(quitar_tildes)

  reemplazos = {
      'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
      'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
      'ARTES': ['ARTE', 'ARTES','BELLAS','ANIMACION'],
      'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
      'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
      'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN', 'COMUNICACIÖN'],
      'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
      'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
      'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
      'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
      'ECONOMIA': ['ECONOM¿A'],
      'GEOLOGIA': ['GEOLOG¿A'],
      'GERENCIA': ['GERONTOLOGIA'],
      'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA','AGROINDUSTRIA'],
      'INSTRUMENTACION': ['INTRUMENTACION'],
      'MATEMATICAS': ['MATEMATICA'],
      'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
      'PSICOLOGIA': ['PSICOLOGA'],
      'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
      'MUSICA': ['MUSICA-'],
      'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
      'GESTION': ['GESTI¿N'],
      'NEGOCIOS' : ['3°'],
      'DEPORTE' : ['ACTIVIDAD'],
      'LICENCIATURA' : ['Licenciatura','LICENC.EN'],
      'LENGUAS' :['LENGUAJES']

  }

  for key, values in reemplazos.items():
      df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].replace(values, key)

  ##Eliminar columnas

  df = df.drop(labels =['ESTU_PRIVADO_LIBERTAD','FAMI_TIENEINTERNET.1', 'coef_1', 'coef_2', 'coef_3', 'coef_4'],axis= 1)

  if 'RENDIMIENTO_GLOBAL' in df.columns:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']
  else:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE']

  ##Convertir a binario

  def convertir_num(valor):
      if valor == 'Si':
          return 1
      elif valor == 'No':
          return 0
      else:
          return valor

  # Aplicar la función a todas las columnas del DataFrame
  df = df.applymap(convertir_num)

  ##Imputar Datos

  df['ESTRATO'] = df['ESTRATO'].str.replace('Estrato ', '')

  columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                        'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

  for columna in columnas_binarias:
      moda = df[columna].mode()[0]
      df[columna].fillna(moda, inplace=True)


  ModaEstrato = df['ESTRATO'].mode()[0]  # Obtener la moda
  df['ESTRATO'].fillna(ModaEstrato, inplace=True)

  ModaPadre = df['EDUCACION_PADRE'].mode()[0]
  df['EDUCACION_PADRE'].fillna(ModaPadre, inplace=True)

  ModaMadre = df['EDUCACION_MADRE'].mode()[0]
  df['EDUCACION_MADRE'].fillna(ModaMadre, inplace=True)

  ValorMatricula = df['VALOR_MATRICULA'].mode()[0]
  df['VALOR_MATRICULA'].fillna(ValorMatricula, inplace=True)

  HorasTrabaja = df['HORAS_TRABAJA'].mode()[0]
  df['HORAS_TRABAJA'].fillna(HorasTrabaja, inplace=True)

  ##Convertir columnas categoricas
  Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO","EDUCACION_MADRE","EDUCACION_PADRE"]

  df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

  valores_rendimiento = {'alto': 4,'medio-alto': 3,'medio-bajo':2, 'bajo': 1}
  df= df.replace(valores_rendimiento)

  ##Convertir a Onehot
  encoder = OneHotEncoder(sparse_output=False)

  encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA','PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE']])

  feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA','PERIODO', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'])

  one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

  df = pd.concat([df, one_hot_encoded_df], axis=1)

  df = df.drop(labels =['HORAS_TRABAJA', 'PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'],axis= 1)

  return df

ESTU_PRGM_ACADEMICO ['ACUICULTURA' 'ADMINISTRACION' 'AGRONOMIA' 'ANTROPOLOGIA' 'ARCHIVISTICA'
 'ARQUEOLOGIA' 'ARQUITECTURA' 'ARTES' 'ASTRONOMIA' 'BACTERIOLOGIA' 'BANCA'
 'BIBLIOTECOLOGIA' 'BIOINGENIERIA' 'BIOLOGIA' 'BIOQUIMICA' 'BIOTECNOLOGIA'
 'CIENCIA' 'CIENCIAS' 'CINE' 'COMERCIO' 'COMPUTACION' 'COMUNICACION'
 'CONSERVACION' 'CONSTRUCCION' 'CONTADURIA' 'CREACION' 'CRIMINALISTICA'
 'CULINARIA' 'CULTURA' 'DANZA' 'DEPORTE' 'DERECHO' 'DESARROLLO'
 'DIRECCION' 'DISEÑO' 'ECOLOGIA' 'ECONOMIA' 'ENFERMERIA' 'ENTRENAMIENTO'
 'ESPAÑOL' 'ESTADISTICA' 'ESTUDIOS' 'FARMACIA' 'FILOLOGIA' 'FILOSOFIA'
 'FINANZAS' 'FISICA' 'FISIOTERAPIA' 'FONOAUDIOLOGIA' 'FORMACION'
 'FOTOGRAFIA' 'GASTRONOMIA' 'GEOCIENCIAS' 'GEOGRAFIA' 'GEOLOGIA'
 'GERENCIA' 'GESTION' 'GOBIERNO' 'HISTORIA' 'HOTELERIA' 'INGENIERIA'
 'INSTRUMENTACION' 'INTERPRETACION' 'INVESTIGACION' 'JURISPRUDENCIA'
 'LENGUAS' 'LETRAS' 'LICENCIATURA' 'LINGUISTICA' 'LITERATURA' 'LOGISTICA'
 'MAESTRO' 'MARKETING' 'MATEMATICAS' 'MEDICINA' 'MICROBIOLOGIA' '

In [ ]:
df = Preprocesar(df)
test_df = Preprocesar(test_df)

/tmp/ipython-input-7-3946954483.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convertir_num)
/tmp/ipython-input-7-3946954483.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(moda, inplace=True)
/tmp/ipython-input-7-3946954483.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because 

In [ ]:
X = df.drop(labels=["RENDIMIENTO_GLOBAL", "ID"], axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Número de clases en y
num_classes = len(label_encoder.classes_)

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Número de unidades igual a clases y softmax para clasificación multiclase

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step - accuracy: 0.4009 - loss: 1.2517 - val_accuracy: 0.4253 - val_loss: 1.2193
Epoch 2/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.4193 - loss: 1.2256 - val_accuracy: 0.4264 - val_loss: 1.2148
Epoch 3/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.4221 - loss: 1.2204 - val_accuracy: 0.4270 - val_loss: 1.2169
Epoch 4/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.4244 - loss: 1.2186 - val_accuracy: 0.4281 - val_loss: 1.2121
Epoch 5/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.4257 - loss: 1.2166 - val_accuracy: 0.4280 - val_loss: 1.2108
Epoch 6/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.4251 - loss: 1.2163 - val_accuracy: 0.4248 - val_loss: 1.2160
Epoch 7/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 87s 3ms/step - accuracy: 0.4271 - loss: 1.2145 - val_accuracy: 0.4291 - val_loss: 1.2119
Epoch 8/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 

In [ ]:
predictions = model.predict(test_df.drop('ID', axis=1))

predicted_classes = tf.argmax(predictions, axis=1) + 1  # Sumar 1 para ajustar el índice de las clases

submit = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predicted_classes})

# Mapear las clases predichas al rango de etiquetas originales
submit['RENDIMIENTO_GLOBAL'] = submit['RENDIMIENTO_GLOBAL'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

submit.to_csv('submission.csv', index=False)
